In [11]:
import numpy as np
import matplotlib.pyplot as plt
import json
import os, sys, glob
from collections import defaultdict

In [ ]:
matches = glob.glob('experiment_comparison/**/[a-zA-Z]*_results.json', recursive=True)
matches = [a for a in matches if "secondary" not in a]

for match in matches:
    print(match)

enxhi_thesis_xstream_cpp\random_tests\results_breastw\breastw_results.json
enxhi_thesis_xstream_cpp\random_tests\results_ionosphere_15\ionosphere_results.json
enxhi_thesis_xstream_cpp\random_tests\results_ionosphere_25\ionosphere_results.json
enxhi_thesis_xstream_cpp\random_tests\results_ionosphere_50\ionosphere_results.json
enxhi_thesis_xstream_cpp\random_tests\results_mammography_25\mammography_results.json
enxhi_thesis_xstream_cpp\results_annthyroid\annthyroid_results.json
enxhi_thesis_xstream_cpp\results_breastw\breastw_results.json
enxhi_thesis_xstream_cpp\results_cardio\cardio_results.json
enxhi_thesis_xstream_cpp\results_ionosphere\ionosphere_results.json
enxhi_thesis_xstream_cpp\results_letter\letter_results.json
enxhi_thesis_xstream_cpp\results_mammography\mammography_results.json
enxhi_thesis_xstream_cpp\results_mnist\mnist_results.json
enxhi_thesis_xstream_cpp\results_optdigits\optdigits_results.json
enxhi_thesis_xstream_cpp\results_pendigits\pendigits_results.json
enxhi_the

In [ ]:
def average_seeds(results_file):
    with open(results_file, 'r') as f:
        results = json.load(f)

    # metric -> model  ->  n_new_trees -> nfeatures -> size -> value
    out_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(list)))))
    
    for n_new_trees in results:
        res_d_0 = results[n_new_trees]
        for seed in res_d_0:
            res_d_1 = res_d_0[seed]
            for model in res_d_1:
                res_d_2 = res_d_1[model]
                for metric in res_d_2:
                    res_d_3 = res_d_2[metric]
                    for nfeatures in res_d_3:
                        res_d_4 = res_d_3[nfeatures]
                        for size in res_d_4:
                            value = res_d_4[size]
                            out_dict[metric][model][n_new_trees][nfeatures][size].append(value)
    
    for metric in out_dict:
        for model in out_dict[metric]:
            for n_new_trees in out_dict[metric][model]:
                for nfeatures in out_dict[metric][model][n_new_trees]:
                    for size in out_dict[metric][model][n_new_trees][nfeatures]:
                        values = out_dict[metric][model][n_new_trees][nfeatures][size]
                        if values:
                            out_dict[metric][model][n_new_trees][nfeatures][size] = np.mean(values)
                        else:
                            out_dict[metric][model][n_new_trees][nfeatures][size] = None
    
    return out_dict

In [14]:
average_seeds(matches[0])  # Example usage, replace with the desired file

defaultdict(<function __main__.average_seeds.<locals>.<lambda>()>,
            {'average_precision': defaultdict(<function __main__.average_seeds.<locals>.<lambda>.<locals>.<lambda>()>,
                         {'fixed': defaultdict(<function __main__.average_seeds.<locals>.<lambda>.<locals>.<lambda>.<locals>.<lambda>()>,
                                      {'n_new_trees:100': defaultdict(<function __main__.average_seeds.<locals>.<lambda>.<locals>.<lambda>.<locals>.<lambda>.<locals>.<lambda>()>,
                                                   {'n_features:7': defaultdict(list,
                                                                {'train_set_size:2': 0.6704368984661782,
                                                                 'train_set_size:3': 0.9403375667412772,
                                                                 'train_set_size:4': 0.9345531702870392,
                                                                 'train_set_size:6': 0.967937022

In [15]:
save_path = "average_results"
for match in matches:
    base_path = os.path.dirname(match)
    file_name = os.path.basename(match)
    save_file = os.path.join(save_path, file_name)
    if not os.path.exists(save_path):
        os.makedirs(save_path, exist_ok=False)

    out_dict = average_seeds(match)
    with open(save_file, 'w', encoding='utf-8') as f:
        json.dump(out_dict, f, indent=4)